In [ ]:
import pandas as pd
import numpy as np
import time
import json
from datetime import datetime
import joblib
import random
import math
import pyarrow as pa 
import ctypes
from tqdm.auto import tqdm 
from scipy.interpolate import interp1d
from math import pi, sqrt, exp
import sklearn,sklearn.model_selection
import torch
from torch import nn,Tensor
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset, SubsetRandomSampler
from sklearn.metrics import average_precision_score
from timm.scheduler import CosineLRScheduler
from pyarrow.parquet import ParquetFile
import gc
import matplotlib.pyplot as plt
plt.style.use("ggplot")

In [ ]:
MAIN_DIR = "/kaggle/input/child-mind-institute-detect-sleep-states/"
TRAIN_SERIES = MAIN_DIR + "train_series.parquet"
TRAIN_EVENTS = MAIN_DIR + "train_events.csv"

In [1]:
class data_reader:
    def __init__(self):
        super().__init__()
        self.names_mapping = {
            "train_events" : {"path" : TRAIN_EVENTS, "is_parquet" : False, "has_timestamp" : True},
            "train_series" : {"path" : TRAIN_SERIES, "is_parquet" : True, "has_timestamp" : True}
        }
        self.valid_names = ["train_events", "train_series"]
    
    def cleaning(self, data):
        "cleaning function : drop na values"
        before_cleaning = len(data)
        print("Number of missing timestamps : ", len(data[data["timestamp"].isna()]))
        data = data.dropna(subset=["timestamp"])
        after_cleaning = len(data)
        print("Percentage of removed steps : {:.1f}%".format(100 * (before_cleaning - after_cleaning) / before_cleaning) )
        return data
    
    @staticmethod
    def reduce_memory_usage(data):
        start_mem = data.memory_usage().sum() / 1024**2
        print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
        for col in data.columns:
            col_type = data[col].dtype    
            if col_type != object:
                c_min = data[col].min()
                c_max = data[col].max()
                if str(col_type)[:3] == 'int':
                    if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                        data[col] = data[col].astype(np.int8)
                    elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                        data[col] = data[col].astype(np.int16)
                    elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                        data[col] = data[col].astype(np.int32)
                    elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                        data[col] = data[col].astype(np.int64)  
                else:
                    if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                        data[col] = data[col].astype(np.float16)
                    elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                        data[col] = data[col].astype(np.float32)
                    else:
                        data[col] = data[col].astype(np.float64)
            else:
                data[col] = data[col].astype('category')

        end_mem = data.memory_usage().sum() / 1024**2
        print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
        print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
        return data
    
   

    def load_data(self, data_name):
        data_props = self.names_mapping[data_name]
        if data_props["is_parquet"]:
            data = pd.read_parquet(data_props["path"])
        else:
            data = pd.read_csv(data_props["path"])
            
        gc.collect()
        print('cleaning')
        data = self.cleaning(data)
        gc.collect()
        data = self.reduce_memory_usage(data)
        return data

In [ ]:
reader = data_reader()
series = reader.load_data(data_name="train_series")
events = reader.load_data(data_name="train_events")

In [ ]:
targets = []
data = []
ids = series.series_id.unique()

for viz_id in tqdm(ids):
    viz_targets = []
    viz_events = events[events.series_id == viz_id]
    v_series = series.loc[(series.series_id == viz_id)].copy().reset_index()
    
    v_series['dt'] = pd.to_datetime(v_series.timestamp, format='%Y-%m-%dT%H:%M:%S%z').astype("datetime64[ns, UTC-04:00]")
    v_series['date'] = v_series['dt'].dt.date
    
    #     Extract full day data 17280 step each step has 5 seconds
    steps_per_day = v_series.groupby(['date'], as_index=False)['step'].count()
    valid_days = steps_per_day[steps_per_day['step'] == 17280]
    viz_series = pd.merge(v_series, valid_days[['date']], on=['date'], how='inner')
    
    for i in range(len(viz_events) - 1):
        if viz_events.iloc[i].event == 'onset' and viz_events.iloc[i + 1].event == 'wakeup' and viz_events.iloc[i].night == viz_events.iloc[i + 1].night:
            start, end = viz_events.timestamp.iloc[i], viz_events.timestamp.iloc[i + 1]

            matching_start_rows = viz_series.loc[viz_series.timestamp == start]
            matching_end_rows = viz_series.loc[viz_series.timestamp == end]
            
            if not matching_start_rows.empty and not matching_end_rows.empty:
                start_id = matching_start_rows.index.values[0]
                end_id = matching_end_rows.index.values[0]
                viz_targets.append((start_id, end_id))
            else:
                print(f"No match found for start timestamp: {start} or end timestamp: {end}")
                continue  # Skip this iteration if no match is found
    
    targets.append(viz_targets)
    data.append(viz_series[['anglez', 'enmo', 'step']])
    

In [ ]:
joblib.dump((targets, data, ids), 'train_data.pkl')
len(data)

In [ ]:
del series
del events
del targets
del data 
del ids
gc.collect()

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# Constants
SIGMA = 720  # 12 hours
SAMPLE_FREQ = 12  # 1 observation per minute
EPOCHS = 5
BS = 1
WARMUP_PROP = 0.2
TRAIN_PROP = 0.9
max_chunk_size = 24*60*12

In [ ]:
def normalize(y):
    mean = y[:,0].mean().item()
    std = y[:,0].std().item()
    y[:,0] = (y[:,0]-mean)/(std+1e-16)
    mean = y[:,1].mean().item()
    std = y[:,1].std().item()
    y[:,1] = (y[:,1]-mean)/(std+1e-16)
    return y

In [ ]:
class CHIDataset(Dataset):
    def __init__(self, file):
        self.targets, self.data, self.ids = joblib.load(file)

    def downsample_features(self, feat, window_size):
        # Ensure feat is of type float64 to prevent overflow
        feat = feat.astype(np.float64)
        
        # Pad the sequence if necessary
        if len(feat) % window_size != 0:
            padding = np.zeros(window_size - (len(feat) % window_size), dtype=np.float64) + feat[-1]
            feat = np.concatenate([feat, padding])
        
        # Reshape into (num_windows, window_size)
        feat = np.reshape(feat, (-1, window_size))
        
        # Calculate mean and standard deviation for each window
        feat_mean = np.mean(feat, axis=1)
        feat_std = np.std(feat, axis=1)
        
        # Stack the mean and standard deviation features
        return np.vstack((feat_mean, feat_std)).T

    def __len__(self):
        return len(self.targets)

    def __getitem__(self, index):
        X = self.data[index][['anglez', 'enmo']].astype(np.float64)
        y = self.targets[index]

        # Generate Gaussian target
        target_guassian = np.zeros((len(X), 2), dtype=np.float64)
        for s, e in y:
            st1, st2 = max(0, s-SIGMA//2), s+SIGMA//2+1
            ed1, ed2 = e-SIGMA//2, min(len(X), e+SIGMA//2+1)
            target_guassian[st1:st2, 0] = self.gauss()[st1-(s-SIGMA//2):]
            target_guassian[ed1:ed2, 1] = self.gauss()[:SIGMA+1-((e+SIGMA//2+1)-ed2)]
        y = target_guassian

        # Fixing SettingWithCopyWarning
        X.loc[:, 'anglez'] = np.abs(X['anglez'])

        # Downsample features for window sizes 12, 360, and 720
        features = []
        max_len = 0
        for window_size in [12, 360, 720]:
            for column in ['anglez', 'enmo']:
                downsampled = self.downsample_features(X[column].values, window_size)
                features.append(downsampled)
                max_len = max(max_len, len(downsampled))

        # Ensure all features have the same length by padding
        for i in range(len(features)):
            if len(features[i]) < max_len:
                padding = np.zeros((max_len - len(features[i]), features[i].shape[1]), dtype=np.float64)
                features[i] = np.vstack((features[i], padding))
                
        # Concatenate all features along the last axis
        X = np.concatenate(features, axis=1)

        # Downsample target
        y = np.dstack([self.downsample_seq(y[:, i], SAMPLE_FREQ) for i in range(y.shape[1])])[0]
        y = normalize(torch.from_numpy(y))

        X = torch.from_numpy(X)
        return X, y


    def gauss(self, n=SIGMA, sigma=SIGMA*0.15):
        # Gaussian distribution function with higher precision
        r = range(-int(n/2), int(n/2)+1)
        return np.array([1 / (sigma * np.sqrt(2*np.pi)) * np.exp(-float(x)**2/(2*sigma**2)) for x in r], dtype=np.float64)

    def downsample_seq(self, feat, downsample_factor=SAMPLE_FREQ):
        # Downsample data with higher precision
        feat = feat.astype(np.float64)
        if len(feat) % downsample_factor != 0:
            padding = np.zeros(downsample_factor - (len(feat) % downsample_factor), dtype=np.float64) + feat[-1]
            feat = np.concatenate([feat, padding])
        feat = np.reshape(feat, (-1, downsample_factor))
        feat_mean = np.mean(feat, axis=1)
        return feat_mean

In [ ]:
train_ds = CHIDataset('/kaggle/working/train_data.pkl')

In [ ]:
class ResidualBiGRU(nn.Module):
    def __init__(self, hidden_size, n_layers=1, bidir=True):
        super(ResidualBiGRU, self).__init__()

        self.hidden_size = hidden_size
        self.n_layers = n_layers

        self.gru = nn.GRU(
            hidden_size,
            hidden_size,
            n_layers,
            batch_first=True,
            bidirectional=bidir,
        )
        
        dir_factor = 2 if bidir else 1
        self.fc1 = nn.Linear(hidden_size * dir_factor, hidden_size * dir_factor * 2)
        self.ln1 = nn.LayerNorm(hidden_size * dir_factor * 2)
        self.fc2 = nn.Linear(hidden_size * dir_factor * 2, hidden_size)
        self.ln2 = nn.LayerNorm(hidden_size)

    def forward(self, x, h=None):
        res, new_h = self.gru(x, h)
        # res.shape = (batch_size, sequence_size, 2*hidden_size)

        res = self.fc1(res)
        res = self.ln1(res)
        res = nn.functional.relu(res)

        res = self.fc2(res)
        res = self.ln2(res)
        res = nn.functional.relu(res)

        # skip connection
        res = res + x

        return res, new_h

In [ ]:
class MultiResidualBiGRU(nn.Module):
    def __init__(self, input_size, hidden_size, out_size, n_layers, bidir=True):
        super(MultiResidualBiGRU, self).__init__()

        self.input_size = input_size
        self.hidden_size = hidden_size
        self.out_size = out_size
        self.n_layers = n_layers

        self.fc_in = nn.Linear(input_size, hidden_size)
        self.ln = nn.LayerNorm(hidden_size)
        self.res_bigrus = nn.ModuleList(
            [
                ResidualBiGRU(hidden_size, n_layers=1, bidir=bidir)
                for _ in range(n_layers)
            ]
        )
        self.fc_out = nn.Linear(hidden_size, out_size)

    def forward(self, x, h=None):
        if h is None:
            h = [None for _ in range(self.n_layers)]

        x = self.fc_in(x)
        x = self.ln(x)
        x = nn.functional.relu(x)

        new_h = []
        for i, res_bigru in enumerate(self.res_bigrus):
            x, new_hi = res_bigru(x, h[i])
            new_h.append(new_hi)

        x = self.fc_out(x)
        return x, new_h  # log probabilities + hidden states

In [ ]:
def plot_history(history, model_path=".", show=True):
    epochs = range(1, len(history["train_loss"]) + 1)

    plt.figure()
    plt.plot(epochs, history["train_loss"], label="Training Loss")
    plt.plot(epochs, history["valid_loss"], label="Validation Loss")
    plt.title("Loss evolution")
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.legend()
    plt.savefig(os.path.join(model_path, "loss_evo.png"))
    if show:
        plt.show()
    plt.close()

    plt.figure()
    plt.plot(epochs, history["lr"])
    plt.title("Learning Rate evolution")
    plt.xlabel("Epochs")
    plt.ylabel("LR")
    plt.savefig(os.path.join(model_path, "lr_evo.png"))
    if show:
        plt.show()
    plt.close()

In [ ]:
train_size = int(TRAIN_PROP * len(train_ds))
valid_size = len(train_ds) - train_size
indices = torch.randperm(len(train_ds))
train_sampler = SubsetRandomSampler(indices[:train_size])
valid_sampler = SubsetRandomSampler(indices[train_size : train_size + valid_size])
steps = train_size*EPOCHS
warmup_steps = int(steps*WARMUP_PROP)

model = MultiResidualBiGRU(input_size=12,hidden_size=64,out_size=2,n_layers=5).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3,weight_decay = 0)
scheduler = CosineLRScheduler(optimizer,t_initial= steps,warmup_t=warmup_steps, warmup_lr_init=1e-6,lr_min=2e-8)

dt = time.time()
model_path = '/kaggle/working/'
history = {
    "train_loss": [],
    "valid_loss": [],
    "valid_mAP": [],
    "lr": [],
}

best_valid_loss = np.inf
criterion = torch.nn.MSELoss()

In [ ]:
def evaluate(model: nn.Module, max_chunk_size: int, loader: DataLoader, device, criterion):
    model.eval()
    valid_loss = 0.0
    y_true_full = torch.FloatTensor([]).half()
    y_pred_full = torch.FloatTensor([]).half()
    for X_batch, y_batch in tqdm(loader, desc="Eval", unit="batch"):
        # as some of the sequences we are dealing with are pretty long, we use a chunk based approach
        y_batch = y_batch.to(device, non_blocking=True)
        pred = torch.zeros(y_batch.shape).to(device, non_blocking=True).half()

        # (re)initialize model's hidden state
        h = None

        # number of chunks for this sequence (we assume batch size = 1)
        seq_len = X_batch.shape[1]
        for i in range(0, seq_len, max_chunk_size):
            X_chunk = X_batch[:, i : i + max_chunk_size].float().to(device, non_blocking=True)

            y_pred, h = model(X_chunk, h)
            h = [hi.detach() for hi in h]
            pred[:, i : i + max_chunk_size] = y_pred.half()
            del X_chunk
            gc.collect()
            
        loss = criterion(
            pred.float(),
            y_batch.float(),
        )
        valid_loss += loss.item()
        del pred,loss
        gc.collect()

    valid_loss /= len(loader)
    y_true_full = y_true_full.squeeze(0)
    y_pred_full = y_pred_full.squeeze(0)
    gc.collect()
    return valid_loss

In [ ]:
train_loader = DataLoader(
    train_ds,
    batch_size=BS,
    sampler=train_sampler,
    pin_memory=True,
    num_workers=1,
)
valid_loader = DataLoader(
    train_ds,
    batch_size=1,
    sampler=valid_sampler,
    pin_memory=True,
    num_workers=1,
)
for epoch in range(1, EPOCHS + 1):
    train_loss = 0.0
    n_tot_chunks = 0
    pbar = tqdm(
        train_loader, desc="Training", unit="batch"
    )
    model.train()
    for step,(X_batch, y_batch) in enumerate(pbar):
        y_batch = y_batch.to(device, non_blocking=True)
        pred = torch.zeros(y_batch.shape).to(device, non_blocking=True)
        optimizer.zero_grad()
        scheduler.step(step+train_size*epoch)
        h = None

        seq_len = X_batch.shape[1]
        for i in range(0, seq_len, max_chunk_size):
            X_chunk = X_batch[:, i : i + max_chunk_size].float()

            X_chunk = X_chunk.to(device, non_blocking=True)

            y_pred, h = model(X_chunk, h)
            h = [hi.detach() for hi in h]
            pred[:, i : i + max_chunk_size] = y_pred
            del X_chunk,y_pred

        loss = criterion(
            normalize(pred).float(),
            y_batch.float(),
        )
        loss.backward()
        train_loss += loss.item()
        n_tot_chunks+=1
        pbar.set_description(f'Training: loss = {(train_loss/n_tot_chunks):.2f}')

        nn.utils.clip_grad_norm_(model.parameters(), max_norm=1e-1)
        optimizer.step()
        del pred,loss,y_batch,X_batch,h
        gc.collect()
    train_loss /= len(train_loader)
    del pbar
    gc.collect()
    ctypes.CDLL("libc.so.6").malloc_trim(0)

    if epoch % 1 == 0:
        valid_loss = evaluate(
            model, max_chunk_size, valid_loader, device, criterion
        )

        history["train_loss"].append(train_loss)
        history["valid_loss"].append(valid_loss)
        history["lr"].append(optimizer.param_groups[0]["lr"])

        if valid_loss < best_valid_loss:
            best_valid_loss = valid_loss
            torch.save(
                model.state_dict(),
                os.path.join(model_path, f"model_best.pth"),
            )

        dt = time.time() - dt
        print(
            f"{epoch}/{EPOCHS} -- ",
            f"train_loss = {train_loss:.6f} -- ",
            f"valid_loss = {valid_loss:.6f} -- ",
            f"time = {dt:.6f}s",
        )
        dt = time.time()

In [ ]:
plot_history(history, model_path=model_path)

In [ ]:
history_path = os.path.join(model_path, "history.json")
with open(history_path, "w", encoding="utf-8") as f:
    json.dump(history, f, ensure_ascii=False, indent=4)